<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Llama_3_1_8B_Instruct_AIAGENT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 OR L4 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
turnulence_df = pd.read_csv('/content/gdrive/MyDrive/datasets/turbulence/REF.csv')

In [3]:
turnulence_df

,Unnamed: 0,REF_U_1,REF_U_2,REF_U_3,REF_tau_11,REF_tau_12,REF_tau_13,REF_tau_22,REF_tau_23,REF_tau_33,...,REF_tauplus_22,REF_tauplus_23,REF_tauplus_31,REF_tauplus_32,REF_tauplus_33,REF_Uplus_1,REF_Uplus_2,REF_Uplus_3,REF_yplus,Case
0,0,0.183148,5.524527e-08,0.000000e+00,8.607882e-03,-7.599257e-06,0.000000e+00,1.508704e-07,0.000000e+00,3.648287e-03,...,1.648931e-08,0.0,0.0,0.0,0.000399,0.060548,1.826393e-08,0.0,0.060548,fp_1000
1,1,0.554677,5.522239e-07,0.000000e+00,5.727882e-02,-9.240743e-05,0.000000e+00,3.399372e-06,0.000000e+00,2.366991e-02,...,3.715329e-07,0.0,0.0,0.0,0.002587,0.183375,1.825637e-07,0.0,0.183376,fp_1000
2,2,0.936817,1.724616e-06,0.000000e+00,1.533795e-01,-3.501041e-04,0.000000e+00,1.554829e-05,0.000000e+00,6.197673e-02,...,1.699343e-06,0.0,0.0,0.0,0.006774,0.309709,5.701532e-07,0.0,0.309713,fp_1000
3,3,1.329845,3.472650e-06,0.000000e+00,3.129591e-01,-1.097912e-03,0.000000e+00,7.115068e-05,0.000000e+00,1.216620e-01,...,7.776382e-06,0.0,0.0,0.0,0.013297,0.439643,1.148049e-06,0.0,0.439660,fp_1000
4,4,1.734088,5.539989e-06,0.000000e+00,5.072728e-01,-2.106934e-03,0.000000e+00,1.497585e-04,0.000000e+00,1.931306e-01,...,1.636779e-05,0.0,0.0,0.0,0.021108,0.573285,1.831505e-06,0.0,0.573321,fp_1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902807,9211,0.005959,-8.866439e-06,1.956330e-06,3.380772e-06,2.847676e-08,2.475930e-09,1.864412e-07,-1.829451e-08,3.345494e-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,squareDuctAve_Re_3500
902808,9212,0.004547,-2.971654e-06,9.090173e-07,2.616207e-06,1.654386e-08,1.419062e-09,6.365150e-08,-7.552760e-09,2.102089e-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,squareDuctAve_Re_3500
902809,9213,0.003354,-6.777688e-07,3.514683e-07,2.089987e-06,9.503438e-09,9.837950e-10,1.963132e-08,-2.862823e-09,1.315723e-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,squareDuctAve_Re_3500
902810,9214,0.002072,-4.948563e-08,5.171945e-08,1.351600e-06,3.031013e-09,6.699116e-10,3.164032e-09,-4.900372e-10,5.482198e-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,squareDuctAve_Re_3500


In [2]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

In [3]:
import os
from sklearn.linear_model import LogisticRegression

class FlightEnvironment:
    def __init__(self, origin, destination, weather):
        self.origin = origin
        self.destination = destination
        self.weather = weather  # Stores predefined weather data

    def get_possible_routes(self):
        """
        Retrieves possible flight routes between the origin and destination.

        In a real-world scenario, this would involve querying a database or API.
        For simplicity, we'll return a few hardcoded routes.

        Returns:
            list: A list of dictionaries representing flight routes.
                 Each dictionary contains flight details (flight_number, departure_time, arrival_time, price).
                 Returns an empty list if no routes are found.
        """
        routes = {
            ("Montreal", "Toronto"): [
                {
                    "flight_number": "AC123",
                    "departure_time": "8:00",
                    "arrival_time": "9:30",
                    "price": 200,
                },
                {
                    "flight_number": "WJ456",
                    "departure_time": "12:00",
                    "arrival_time": "13:30",
                    "price": 250,
                },
            ],
            ("Montreal", "Vancouver"): [
                {
                    "flight_number": "AC789",
                    "departure_time": "9:00",
                    "arrival_time": "14:00",
                    "price": 500,
                }
            ],
        }
        return routes.get((self.origin, self.destination), [])  # Return empty list if no routes found

    def get_weather_data(self, location):
        """
        Retrieves weather data for a given location.

        Simulates fetching weather data from a dictionary.
        In a real-world scenario, this would involve querying a weather API.

        Args:
            location (str): The location for which to retrieve weather data.

        Returns:
            dict: A dictionary containing weather information (weather, wind).
                 Defaults to sunny weather with 5 m/s wind speed if location is not found.
        """
        return self.weather.get(
            location, {"weather": [{"description": "Sunny"}], "wind": {"speed": 5}}
        )

#### class FlightAgent: ####

import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import re

class FlightAgent:
    def __init__(self, environment, turbulence_dataset_path, llama_model_id="meta-llama/Llama-3.1-8B-Instruct"):
        self.environment = environment
        self.turbulence_model = LogisticRegression()
        self.turbulence_dataset_path = turbulence_dataset_path
        self.train_turbulence_model()  # Train the turbulence model during initialization

        # Initialize Llama 3 with padding token fix
        self.llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_id, use_fast=True)
        self.llama_tokenizer.padding_side = "right"
        self.llama_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

        # Initialize Llama 3
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        self.llama_model = AutoModelForCausalLM.from_pretrained(
            llama_model_id,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            quantization_config=bnb_config
        )
        self.llama_model.resize_token_embeddings(len(self.llama_tokenizer))

        self.llama_pipeline = pipeline(
            "text-generation",
            model=self.llama_model,
            tokenizer=self.llama_tokenizer,
            pad_token_id=self.llama_tokenizer.pad_token_id
        )

        self.generation_params = {
            "max_new_tokens": 1024,
            "do_sample": True,
            "top_k": 50,
            "top_p": 0.95,
            "temperature": 0.7,
            "pad_token_id": self.llama_tokenizer.pad_token_id,
        }

    def train_turbulence_model(self):
        """Trains the turbulence prediction model using historical data."""
        turbulence_df = pd.read_csv(self.turbulence_dataset_path)
        turbulence_df['Turbulence'] = (turbulence_df['REF_k'] > turbulence_df['REF_k'].mean()).astype(int)
        features = turbulence_df[[
            "REF_U_1", "REF_U_2", "REF_U_3",
            "REF_tau_11", "REF_tau_12", "REF_tau_13",
            "REF_tau_22", "REF_tau_23", "REF_tau_33"
        ]]
        target = turbulence_df['Turbulence']
        self.turbulence_model.fit(features, target)
        print("Turbulence model trained successfully!")

    def analyze_weather(self):
        """Analyzes weather data for potential risks to flight operations."""
        origin_weather = self.environment.get_weather_data(self.environment.origin)
        destination_weather = self.environment.get_weather_data(self.environment.destination)
        print(f"Weather in {self.environment.origin}: {origin_weather['weather'][0]['description']}")
        print(f"Weather in {self.environment.destination}: {destination_weather['weather'][0]['description']}")
        prompt = f"""
        You are a flight safety expert. Analyze the following weather reports for potential risks to flight operations:
        Origin ({self.environment.origin}): {origin_weather}
        Destination ({self.environment.destination}): {destination_weather}
        Identify any potential hazards or concerns for a flight between these locations. Be specific and concise.
        """
        response = self.llama_pipeline(prompt, **self.generation_params)[0]['generated_text']
        print("Llama 3 weather analysis:", response)
        return response

    def assess_flight_options(self):
        """Assesses and displays available flight options."""
        possible_routes = self.environment.get_possible_routes()
        print("Available flights:")
        for route in possible_routes:
            print(route)

    def recommend_flight_plan(self, origin, destination, flights, weather_conditions, passenger_preferences):
        """Recommends the optimal flight plan based on weather and preferences."""
        if passenger_preferences.get('earliest_flight', False):
            return flights[0]
        if passenger_preferences.get('cheapest_flight', False):
            return min(flights, key=lambda flight: flight['price'])
        if any(keyword in weather_conditions.lower() for keyword in ["thunderstorms", "severe weather"]):
            later_flights = [flight for flight in flights if flight['departure_time'] > flights[0]['departure_time']]
            if later_flights:
                return later_flights[0]
            else:
                print("Warning: Severe weather, but no later flights available.")
        return flights[0]

    def choose_flight(self):
        """Chooses the optimal flight based on analysis and recommendations using Llama 3."""
        # Observe:
        weather_analysis_text = self.analyze_weather()  # Get weather analysis text
        # Orient:
        self.assess_flight_options()
        # Decide:
        possible_routes = self.environment.get_possible_routes()  # Get possible routes
        if not possible_routes:
            return "No flights found."

        # Use Llama 3 for flight recommendation with improved prompt and weather analysis
        prompt = f"""
        You are a flight booking agent. Consider the following information:
        - Origin: {self.environment.origin}
        - Destination: {self.environment.destination}
        - Available flights: {possible_routes}
        - Weather conditions: {weather_analysis_text}
        - Passenger preferences: (not included in this example)

        Recommend the optimal flight plan, taking into account safety and convenience.
        Provide the flight number of the recommended flight ONLY.
        """
        response = self.llama_pipeline(prompt, **self.generation_params)[0]['generated_text']

        # Extract the flight number using regex (for robustness)
        flight_number_match = re.search(r"([A-Z]{2}\d{3,4})", response)  # Improved regex
        if flight_number_match:
            chosen_flight = flight_number_match.group(1)  # Extract the flight number
            print(f"Llama 3 recommended flight: {chosen_flight}")
        else:
            print("Could not extract flight number from Llama 3's response. Using fallback.")
            chosen_flight = self.recommend_flight_plan(
                self.environment.origin, self.environment.destination,
                possible_routes, weather_analysis_text, {}
            )  # Use fallback if extraction fails

        return chosen_flight # returns flight Number only

In [4]:
# Initialize environment and agent with weather data
environment = FlightEnvironment(
    origin="Montreal",
    destination="Toronto",
    weather={"Montreal": {"weather": [{"description": "Cloudy"}], "wind": {"speed": 10}},
             "Toronto": {"weather": [{"description": "Rainy"}], "wind": {"speed": 15}}}
)


# Provide the path to the turbulence dataset when creating the FlightAgent instance
agent = FlightAgent(environment, '/content/gdrive/MyDrive/datasets/turbulence/REF.csv')

Turbulence model trained successfully!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
Device set to use cuda:0


In [5]:
# Run the agent
print('\n')
print("Running the agent...")
print('\n\n')
print("Weather analysis:")
print(agent.analyze_weather())
agent.assess_flight_options()
print('\n\n')

print("Llama 3 flight recommendation:")
print('\n\n')
chosen_flight = agent.choose_flight()
print("Chosen flight:")
print(chosen_flight)
print('\n\n')



Running the agent...



Weather analysis:
Weather in Montreal: Cloudy
Weather in Toronto: Rainy
Llama 3 weather analysis: 
        You are a flight safety expert. Analyze the following weather reports for potential risks to flight operations:
        Origin (Montreal): {'weather': [{'description': 'Cloudy'}], 'wind': {'speed': 10}}
        Destination (Toronto): {'weather': [{'description': 'Rainy'}], 'wind': {'speed': 15}}
        Identify any potential hazards or concerns for a flight between these locations. Be specific and concise.
        


```python
# Define the weather reports
origin_weather = {'weather': [{'description': 'Cloudy'}], 'wind': {'speed': 10}}
destination_weather = {'weather': [{'description': 'Rainy'}], 'wind': {'speed': 15}}

# Define the potential hazards and concerns
hazards = [
    # Potential hazard: Reduced visibility due to rain at destination
    f"Reduced visibility due to rain at destination with wind speed {destination_weather['wind']['speed']} knots"